# initialize 

In [10]:
import cv2
import numpy as np

In [11]:
# Court dimensions in meters
court_width = 10.97
court_length = 23.77

# Load video
video_path = 'video/755B5535-4757-479D-97D7-189B7EBB19E9.mov'
cap = cv2.VideoCapture(video_path)

[ WARN:0@1414.305] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@1414.305] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin2 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@1414.306] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@1414.306] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/v

# corner selector

In [3]:

# Function to calculate court coordinates
def calculate_court_coordinates(frame, corners):
    height, width = frame.shape[:2]

    # Define court corners in pixel coordinates
    court_corners = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

    # Compute perspective transform matrix
    M, _ = cv2.findHomography(corners, court_corners)

    # Transform court dimensions to pixel coordinates
    court_pixel_width = int(width * court_width / court_length)
    court_pixel_height = int(height * court_length / court_width)

    # Warp court corners to get court coordinates
    court_coordinates = cv2.perspectiveTransform(np.array([[[0, 0]], [[court_pixel_width, 0]],
                                                           [[court_pixel_width, court_pixel_height]],
                                                           [[0, court_pixel_height]]], dtype=np.float32), M)

    return court_coordinates.squeeze()

# Select court corners
def select_court_corners(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP and len(corners) < 4:
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        corners.append([x, y])

# Initialize court corners
corners = []

# Prompt user to select court corners
print("Select court corners in the order: top left, top right, bottom right, bottom left.")
print("Press 'Esc' key to exit after selecting the corners.")

while True:
    ret, frame = cap.read()

    if not ret:
        break

    cv2.imshow("Select Court Corners", frame)
    cv2.setMouseCallback("Select Court Corners", select_court_corners)

    if cv2.waitKey(1) == 27 or len(corners) == 4:
        break

# Calculate court coordinates
court_coordinates = calculate_court_coordinates(frame, corners)

# Print court coordinates
print("Court Coordinates:")
for coordinate in court_coordinates:
    print(coordinate)

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()


Select court corners in the order: top left, top right, bottom right, bottom left.
Press 'Esc' key to exit after selecting the corners.


AttributeError: 'NoneType' object has no attribute 'shape'

# manually corners coordinates

In [27]:
# Function to calculate court coordinates
def calculate_court_coordinates(frame, corners):
    height, width = frame.shape[:2]

    # Define court corners in pixel coordinates
    court_corners = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

    # Compute perspective transform matrix
    M, _ = cv2.findHomography(corners, court_corners , cv2.RANSAC, 5.0)

    # Transform court dimensions to pixel coordinates
    court_pixel_width = int(width * court_width / court_length)
    court_pixel_height = int(height * court_length / court_width)

    # Warp court corners to get court coordinates
    court_coordinates = cv2.perspectiveTransform(np.array([[[0, 0]], [[court_pixel_width, 0]],
      [[court_pixel_width,court_pixel_height]],
      [[0, court_pixel_height]]], dtype=np.float32), M)

    return court_coordinates.squeeze()

# Manual court corner entry
def enter_court_corners():
    corners = []
    print("Enter court corners in the order: top left, top right, bottom right, bottom left.")
    print("Enter the coordinates as 'x,y' and press Enter.")

    for i in range(4):
        while True:
            coordinates = input(f"Enter corner {i+1} coordinates: ")
            try:
                x, y = map(int, coordinates.split(","))
                corners.append([x, y])
                break
            except ValueError:
                print("Invalid input. Please enter the coordinates as 'x,y'.")

    return corners

# Manual entry of court corners
# corners = enter_court_corners()
corners = [[380,178] , [778,178] , [185,564] , [982,564]]

# Calculate court coordinates
ret, frame = cap.read()
court_coordinates = calculate_court_coordinates(frame, np.float32(corners))

# Print court coordinates
print("Court Coordinates:")
for coordinate in court_coordinates:
    print(coordinate)


Court Coordinates:
[-124.28863  278.82315]
[520.255   278.82315]
[595.03284 529.4526 ]
[752.6714 529.4526]


In [28]:
# Draw lines on the court
court_lines = [(tuple(court_coordinates[0].astype(int)), tuple(court_coordinates[1].astype(int))),  # Top line
               (tuple(court_coordinates[1].astype(int)), tuple(court_coordinates[2].astype(int))),  # Right line
               (tuple(court_coordinates[2].astype(int)), tuple(court_coordinates[3].astype(int))),  # Bottom line
               (tuple(court_coordinates[3].astype(int)), tuple(court_coordinates[0].astype(int)))]  # Left line


# Draw court lines on the video frames
while ret:
    for line in court_lines:
        cv2.line(frame, line[0], line[1], (0, 255, 0), 2)

    cv2.imshow("Video with Court Lines", frame)
    if cv2.waitKey(1) == 27:
        break

    ret, frame = cap.read()

# Release video capture
cap.release()
cv2.destroyAllWindows()

In [24]:
court_lines[0][0]

(-124.28863, 278.82315)

In [29]:
cap.release()
cv2.destroyAllWindows()

: 

[[380,178] , [778,178] , [185,564] , [982,564]]